In [6]:
import praw
import pandas as pd
from praw.models import MoreComments

# #URL to access the app needed to scrape the data off the Berkeley subreddit
# # https://www.reddit.com/prefs/apps


# #PLAN
# #Scrape the data off the Berkeley subreddit about consulting clubs
# #Organize that data using the pandas library
# #Create chatbot that utilizes natural language processing that will give users feedback
# #about each consulting club here at Berkeley. 

reddit_read_only = praw.Reddit(client_id = "QlBfNfxQ3e_MGP9RkaOQig",
                               client_secret = "SpLjOwYdQPU4z1wqcXBjVl_7DnUIZg",
                               user_agent = "Berkeley_Consulting")

subreddit = reddit_read_only.subreddit("berkeley")

# df1 = pd.DataFrame({
#     'post_id': ['p1', 'p2', 'p3'],
#     'title': ['Title A', 'Title B', 'Title C'],
#     'author': ['user1', 'user2', 'user3']
# })

# df2 = pd.DataFrame({
#     'comment_id': ['c1', 'c2', 'c3', 'c4', 'c5'],
#     'post_id': ['p1', 'p1', 'p2', 'p2', 'p4'],  # Note: p4 doesn't exist in df1
#     'comment': ['Great post!', 'Thanks!', 'Interesting', 'I agree', 'Nice!']
# })
# outer_merged = df1.merge(df2, on='post_id', how='outer')
# print(outer_merged)

In [7]:
consulting_posts = subreddit.search('consulting')

posts_dict = {'Title': [], 'Post Text': [], 'ID': [], 'Score': [], 'Total Comments': [], 'Post URL': []}
comments_dict = {'Comment': [], 'Score': []}

In [8]:
for post in consulting_posts:
    #The title of the post
    posts_dict['Title'].append(post.title)
    #The text inside of the post
    posts_dict['Post Text'].append(post.selftext)
    #Unique ID of each post
    posts_dict['ID'].append(post.id)
    #The scoure of a post
    posts_dict['Score'].append(post.score)
    #Total number of comments inside the post
    posts_dict['Total Comments'].append(post.num_comments)
    #URL of each post
    posts_dict['Post URL'].append(post.url)

consulting_club_posts = pd.DataFrame(posts_dict)
# print(consulting_club_posts)
consulting_club_posts

,Title,Post Text,ID,Score,Total Comments,Post URL
0,What are consulting clubs actually looking for?,I am an incoming freshman at Haas next year an...,1kvi9jt,45,55,https://www.reddit.com/r/berkeley/comments/1kv...
1,Asian monoculture in consulting clubs,Over the past month on this subreddit there ha...,16o5u8z,271,112,https://www.reddit.com/r/berkeley/comments/16o...
2,Rejected from consulting club after having bee...,titles kinda self explanatory lol. didn’t even...,1na99k8,73,22,https://www.reddit.com/r/berkeley/comments/1na...
3,Tech PM blocks all “.berkeley.edu” e-mails bc ...,Consulting clubs making us look bad smh,19d8ceg,556,43,https://i.redd.it/em6d28vzh2ec1.jpeg
4,Why are some of these Business/Consulting Club...,My friend and I were enjoying dinner in the so...,16jsy19,368,55,https://www.reddit.com/r/berkeley/comments/16j...
...,...,...,...,...,...,...
95,got into a consulting club that has nothing to...,do i send it? do i not? help.,sn6pop,25,5,https://www.reddit.com/r/berkeley/comments/sn6...
96,another bear rejected by consulting clubs,I know it's just a club but man I wanna cry i ...,piudil,33,7,https://www.reddit.com/r/berkeley/comments/piu...
97,There are 150-odd chapters of Chinese Students...,,69fs7e,50,22,https://www.nytimes.com/2017/05/04/us/chinese-...
98,NEW! - Stress Management Consultations,Stress Management Consultations at the UHS Car...,zdf1h4,6,0,https://www.reddit.com/r/berkeley/comments/zdf...


In [9]:
#Need to figure out a way to get the URL of each and every post
for i in posts_dict['Post URL']:
    if 'comments' in i:
        submission = reddit_read_only.submission(url = i)
    else:
        continue
    for comment in submission.comments:
        if type(comment) == MoreComments:
            continue
        comments_dict['Comment'].append(comment.body)
        comments_dict['Score'].append(comment.score)

# print(post_comments[0]) #This only prints one comment right now for one URL. Need to get as many comments as possible for one given URL.

comments_df = pd.DataFrame(comments_dict)
comments_df

#The comments right now are all of the comments from the post that are not more commments
#They are not marked in any way so they are just random pieces of information
#Need to find a way to associate either the post title or the post URL with the post comments

,Comment,Score
0,Being friends with other people inside the clu...,146
1,These are just glorified personality contests.,119
2,https://preview.redd.it/abja5zpt613f1.jpeg?wid...,125
3,this sounds callous but I legitimately feel as...,43
4,They r overrated af,26
...,...,...
667,Hey . Pm me . Hablo español amiga,0
668,"You might know this one then, what *is* consul...",1
669,"Hey there, just went through the other side of...",2
670,Hi! I went through all the interview rounds bu...,1


In [10]:
import ollama
embed_model = 'hf.co/CompendiumLabs/bge-base-en-v1.5-gguf'
lang_model = 'hf.co/bartowski/Llama-3.2-1B-Instruct-GGUF'

print("Pulling embedding model...")
ollama.pull(embed_model)

print("Pulling language model...")
ollama.pull(lang_model)

print("All models pulled successfully!")

Pulling embedding model...
Pulling language model...
All models pulled successfully!


In [11]:
consulting_club_posts['embedding'] = consulting_club_posts['Post Text'].apply(
    lambda t: ollama.embed(model=embed_model, input=t)['embeddings'])

# Attempt to weight embeddings by multiplying by a coefficient calculated by the score, sigmoid function
# Didn't work, can't multiply floats to pandas column
# consulting_club_posts['embedding'].apply(lambda x: consulting_club_posts['Score'].apply(lambda x: 1 + 1/(1+20*pow(2, -x/100))))

In [12]:
def cosine_similarity(a, b):
    if len(a) < 1 or len(b) < 1 or len(a[0]) != len(b[0]):
        return 0
    dot_prod = sum([x * y for x, y in zip(a[0], b[0])])
    mag_a = pow(sum([pow(x, 2) for x in a[0]]), 0.5)
    mag_b = pow(sum([pow(y, 2) for y in b[0]]), 0.5)
    return dot_prod / (mag_a * mag_b)

In [13]:
def retrieve_data(query, n=3):
    query_embed = ollama.embed(model=embed_model, input=query)['embeddings']
    similarities = consulting_club_posts['embedding'].apply(lambda x: cosine_similarity(query_embed, x))
    return similarities.nlargest(n)

In [14]:
input_query = input('Ask me a question: ')
retrieved_knowledge = retrieve_data(input_query)


instruction_prompt = f'''You are a helpful chatbot.
Use only the following pieces of context to answer the question. Don't make up any new information:
{'\n'.join([f' - {consulting_club_posts.loc[i, 'Post Text']}' for i in retrieved_knowledge.index])}
'''

print(instruction_prompt)

You are a helpful chatbot.
Use only the following pieces of context to answer the question. Don't make up any new information:
 - I am an incoming freshman at Haas next year and plan on joining a consulting club. I know a lot of these clubs have acceptance rates under 5% but what actually sets the 5% apart from the other 95%? Is it behavioral? Personality? Based on accomplishments? Grades even?
 - Over the past month on this subreddit there have been numerous posts about alienation from minority folk that I resonate with as one myself and I'd like to share my own experiences in regards to consulting club culture.

Why is there a legit vacuum of asian members in each of these 'exclusive' and 'prestigious' consulting clubs? Like seriously: look at the club photos on any of their site and it's very very monolithic. Take a look 3 club sites I thought of at the top of my head:

[https://www.voyagerconsulting.org/](https://www.voyagerconsulting.org/)

[https://dssberkeley.com/](https://dssbe

In [15]:
stream = ollama.chat(
  model=lang_model,
  messages=[
    {'role': 'system', 'content': instruction_prompt},
    {'role': 'user', 'content': input_query},
  ],
  stream=True,
)

# print the response from the chatbot in real-time
print('Chatbot response:')
for chunk in stream:
  print(chunk['message']['content'], end='', flush=True)


Chatbot response:
It seems like you've been following the subreddit discussion about consulting club culture and the lack of diversity within some clubs.

To address your question, the 5% acceptance rate in consulting clubs is not necessarily based on meritocracy or behavioral traits. Rather, it's often due to a combination of factors, including:

1. **Social influence**: As you've noticed, most clubs have an existing member base, which can lead to self-selecting for certain demographics.
2. **Networking**: Established members may have existing relationships with alumni or other members from similar backgrounds, making it easier to get in.
3. **Perceived exclusivity**: The image of a prestigious consulting club like Voyager Consulting, DSS Berkeley, and PBL Berkeley may create an impression that only certain groups are welcome.
4. **Lack of diversity in applicant pools**: These clubs often have predominantly white or Asian applications, which can lead to concerns about bias against und